In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
from sklearn.metrics import confusion_matrix
import pickle as pkl

In [2]:
with open('q2.pickle', 'rb') as handle:
    b = pkl.load(handle)
    X,y=b
    X=np.array(X)
    y=np.array(y)

X0=np.array([list(X[i]) for i in range(len(y)) if y[i]==0])
X1=np.array([list(X[i]) for i in range(len(y)) if y[i]==1])

prior0=len(X0)/len(X)
prior1=len(X1)/len(X)

# prior0,prior1
len(X1)

140000

In [3]:
def calculate_cov(X):
    X=np.array(X)
    N, M = X.shape
    # Calculate the covariance matrix
    cov = np.zeros((M, M))
    for i in range(M):
        # Mean of column "i"
        mean_i = np.sum(X[:, i]) / N
        for j in range(M):
            # Mean of column "j"
            mean_j = np.sum(X[:, j]) / N
            # Covariance between column "i" and column "j"
            cov[i, j] = np.sum((X[:, i] - mean_i) * (X[:, j] - mean_j)) / N
    return cov

cov=calculate_cov(X1)
cov

array([[ 3.99402911e+00, -8.41400879e-04],
       [-8.41400879e-04,  7.97419232e+00]])

In [4]:
def max_likelihood(X):
    mean=np.mean(X,axis=0)
#     conv=np.cov(X,rowvar=False)
    cov=calculate_cov(X)
    return mean,cov


mean0,cov0=max_likelihood(X0)
mean1,cov1=max_likelihood(X1)


mus=np.array([mean0,mean1])
sigmas=np.array([cov0,cov1])
priors=np.array([prior0,prior1])
print(f"{mus}\n\n,{sigmas}\n\n,{priors},\n\n{X[100]}")

[[1.0027023  1.00536381]
 [3.99859576 4.00311974]]

,[[[ 9.99332505e-01  2.93812690e-03]
  [ 2.93812690e-03  9.95403756e-01]]

 [[ 3.99402911e+00 -8.41400879e-04]
  [-8.41400879e-04  7.97419232e+00]]]

,[0.3 0.7],

[2.57799777 2.93655184]


In [5]:
class multi_case_1():
    def __init__ (self,mus,sigmas,priors):
        self.mus=mus
        self.sigmas=sigmas
        self.priors=priors
        values=self.calculate_case_1()
        print(f"Paramter Values calculated: ", values)
        return None
    
    def calculate_case_1(self):
        mu_diff=(self.mus[0]-self.mus[1])
        self.w= mu_diff
        self.x0= 1/2 * (self.mus[0]+self.mus[1]) - (self.sigmas[0][0]**2)/((np.linalg.norm(mu_diff))**2)*(math.log(priors[0]/priors[1])*mu_diff)
        return self.w,self.x0
    
    def decision_boundry_fn(self,x):
        return np.matmul(self.w.T, (x-self.x0))
    
    def predict(self,X):
        op_data=[np.matmul(self.w.T, (x-self.x0)) for x in X]
        to_ret=[0 if x>0 else 1 for x in op_data]
        return np.array(to_ret)

In [6]:
model1=multi_case_1(mus,sigmas,priors)

Paramter Values calculated:  (array([-2.99589346, -2.99775592]), array([2.35951564, 2.50424055]))


In [7]:
sigma_inv=np.linalg.inv(sigmas[0])
mu_diff=mus[0]-mus[1]


In [8]:
class multi_case_2():
    def __init__ (self,mus,sigmas,priors):
        self.mus=mus
        self.sigmas=sigmas
        self.priors=priors
        values=self.calculate_case_2()
        print(f"Paramter Values calculated: ", values)
        return None
    
    def calculate_case_2(self):
        sigma_inv=np.linalg.inv(self.sigmas[0])
        mu_diff=self.mus[0]-self.mus[1]
        self.w=np.dot(sigma_inv,mu_diff)
        self.x0=((self.mus[0]+self.mus[1])/2)-np.log(self.priors[0]/priors[1])*mu_diff/(np.matmul(np.matmul(mu_diff.T,sigma_inv),mu_diff))
        return self.w,self.x0
    
    def decision_boundry_fn(self,x):
        return np.matmul(self.w.T, (x-self.x0))
    
    def predict(self,X):
        op_data=[np.matmul(self.w.T, (x-self.x0)) for x in X]
        to_ret=[0 if x>0 else 1 for x in op_data]
        return np.array(to_ret)

In [9]:
model2=multi_case_2(mus,sigmas,priors)

Paramter Values calculated:  (array([-2.98906611, -3.00277515]), array([2.35928448, 2.36278934]))


In [10]:
class multi_case_3():
    def __init__ (self,mus,sigmas,priors):
        self.mus=mus
        self.sigmas=sigmas
        self.priors=priors
        values=self.calculate_case_3_constants()
        print(f"Paramter Values calculated: {self.W12}\n{self.W22}\n{self.w11}\n{self.w21}\n{self.w10}\n{self.w20}")
        
        return None
    
    def calculate_case_3_constants(self):
        dimension=len(self.mus[0])
        print(self.sigmas.shape)
        if (dimension,dimension) != self.sigmas[0].shape:
            return NameError("Dimension mismatch")
        mu1=self.mus[0]
        mu2=self.mus[1]
        sigma1=self.sigmas[0]
        sigma2=self.sigmas[1]
        sigma1_inv=np.linalg.inv(sigma1)
        sigma2_inv=np.linalg.inv(sigma2)
        prior1=self.priors[0]
        prior2=self.priors[1]
        self.W12=-1/2*sigma1_inv
        self.W22=-1/2*sigma2_inv
        self.w11=np.matmul(sigma1_inv,mu1)
        self.w21=np.matmul(sigma2_inv,mu2)
        self.w10=-1/2*np.matmul(np.matmul(mu1.T,sigma1_inv),mu1) - 1/2*np.log(np.linalg.det(sigma1)) + np.log(priors[0])
        self.w20=-1/2*np.matmul(np.matmul(mu2.T,sigma2_inv),mu2) - 1/2*np.log(np.linalg.det(sigma2)) + np.log(priors[1])
        return self.W12,self.W22,self.w11,self.w21,self.w10,self.w20
    
    def decision_boundry_fn(self,x):
        return np.matmul(np.matmul(x.T,(self.W12-self.W22)), x) + np.matmul((self.w11.T-self.w21.T),x) + (self.w10-self.w20) 
    
    def predict(self,X):
        op_data=[np.matmul(np.matmul(x.T,(self.W12-self.W22)), x) + np.matmul((self.w11.T-self.w21.T),x) + (self.w10-self.w20) for x in X]
        to_ret=[0 if x>0 else 1 for x in op_data]
        return np.array(to_ret)
        

In [11]:
model3=multi_case_3(mus,sigmas,priors)

(2, 2, 2)
Paramter Values calculated: [[-0.50033831  0.00147685]
 [ 0.00147685 -0.50231309]]
[[-1.25186872e-01 -1.32091552e-05]
 [-1.32091552e-05 -6.27022764e-02]]
[1.00041122 1.00705314]
[1.00124915 0.50211508]
-2.20911589833643
-5.0939890629151146


In [12]:
y_true=y
y_pred=model3.predict(X)
print(y_pred.shape)

def misclf_rate(y_true, y_pred):
    y_true=list(y_true)
    y_pred=list(y_pred)
    total_class1 = y_true.count(0)
    total_class2 = y_true.count(1)
    misclf_class1 = 0
    misclf_class2 = 0    
    tn, fp, fn, tp = confusion_matrix(y_true,y_pred).ravel()    
    total_misclassification_rate = (fp+fn)/(tn+fp+fn+tp)   
    for i in range(len(y_true)):
        if y_true[i] != y_pred[i]:
            if y_true[i] == 0:
                misclf_class1 += 1
            elif y_true[i] == 1:
                misclf_class2 += 1   
            
    class1_misclf_rate = misclf_class1/total_class1
    class2_misclf_rate = misclf_class2/total_class2    
    return class1_misclf_rate,class2_misclf_rate,total_misclassification_rate

c1_mis,c2_mis,tot_mis=misclf_rate(y_true,y_pred)
c1_mis,c2_mis,tot_mis

(200000,)


(0.08348333333333334, 0.07414285714285715, 0.076945)

In [15]:
def error_rate_for(percentage=0.01,X=X,y=y):
    np.random.seed(35)
    
    train_i=np.random.choice(range(len(X)),int(len(X)*0.01*percentage),replace=False)
    train_X=[X[i] for i in train_i]
    train_y=[y[i] for i in train_i]
    X0=np.array([list(train_X[i]) for i in range(len(train_y)) if train_y[i]==0])
    X1=np.array([list(train_X[i]) for i in range(len(train_y)) if train_y[i]==1])

    prior0=len(X0)/len(train_y)
    prior1=len(X1)/len(train_y)
    print("X0,X1,prior0,prior1",len(X0),len(X1),prior0,prior1)
    
    mean0,cov0=max_likelihood(X0)
    mean1,cov1=max_likelihood(X1)

    mus=np.array([mean0,mean1])
    sigmas=np.array([cov0,cov1])
    priors=np.array([prior0,prior1])
    print(f"mus={mus}\n\n,sigmas={sigmas}\n\n,priors={priors}")
    
    if np.array_equal(cov0,cov1):
        if cov0[0][0]==cov0[1][1] and cov0[0][1]==0 and cov0[1][0]==0:
            model=multi_case_1(mus,sigmas,priors)
        else:
            model=multi_case_2(mus,sigmas,priors)
    else:
        model=multi_case_3(mus,sigmas,priors)
        
    y_pred=model.predict(X)
    print("prediction_shape vs y_true shape:",y_pred.shape, "   ", y.shape)
    
    c1_mis,c2_mis,tot_mis=misclf_rate(y_true=y,y_pred=y_pred)
    
    return c1_mis,c2_mis,tot_mis

In [21]:
error_rate_for(percentage=0.01,X=X,y=y)

X0,X1,prior0,prior1 8 12 0.4 0.6
mus=[[0.89199261 1.26567805]
 [3.34740232 3.46935637]]

,sigmas=[[[ 1.12063254 -0.18333451]
  [-0.18333451  0.83817559]]

 [[ 3.34349589  1.31361533]
  [ 1.31361533  5.7548532 ]]]

,priors=[0.4 0.6]
(2, 2, 2)
Paramter Values calculated: [[-0.46273515 -0.10121426]
 [-0.10121426 -0.61867235]]
[[-0.1642766   0.03749813]
 [ 0.03749813 -0.09544259]]
[1.081722   1.74664478]
[0.83961094 0.41120607]
-2.7422245942263315
-3.906792978315614
prediction_shape vs y_true shape: (200000,)     (200000,)


(0.17656666666666668, 0.05275, 0.089895)